In [1]:
#!pip install nbimporter

In [2]:
import sqlite3
import pandas as pd
import nbimporter
from final_scores import calculate_final_driver_places 

conn = sqlite3.connect('formula1.db')

drivers_final_positions = calculate_final_driver_places(conn)

print(drivers_final_positions)

drivers_final_positions.to_csv("drivers_standing.csv", index=False)

      year  driverId      driver_name  points  place
73    1950       642      Nino Farina    30.0      1
66    1950       579      Juan Fangio    27.0      2
72    1950       786    Luigi Fagioli    24.0      3
70    1950       627     Louis Rosier    13.0      4
56    1950       647   Alberto Ascari    11.0      5
...    ...       ...              ...     ...    ...
3133  2022       855      Guanyu Zhou     6.0     18
3136  2022       848  Alexander Albon     4.0     19
3139  2022       849  Nicholas Latifi     2.0     20
3145  2022       856    Nyck de Vries     2.0     20
3140  2022       807  Nico Hülkenberg     0.0     22

[3146 rows x 5 columns]


In [3]:
from outlier_functions import percentile_outliers

outliers = percentile_outliers(drivers_final_positions)

print(outliers)

    driverId  year place      driver_name  points
0        642  1954     8      Nino Farina     6.0
1        579  1958    14      Juan Fangio     7.0
2        627  1954    26     Louis Rosier     0.0
3        627  1955    26     Louis Rosier     0.0
4        647  1955    26   Alberto Ascari     0.0
..       ...   ...   ...              ...     ...
489      844  2018    13  Charles Leclerc    39.0
490      848  2022    19  Alexander Albon     4.0
491      846  2019    11     Lando Norris    49.0
492      847  2019    20   George Russell     0.0
493      849  2020    21  Nicholas Latifi     0.0

[494 rows x 5 columns]


In [4]:
from outlier_functions import remove_outliers

drivers_without_outliers = remove_outliers(drivers_final_positions, outliers)

print(drivers_without_outliers)

      year  driverId          driver_name  points  place
556   1955       579          Juan Fangio   40.00      1
544   1955       475        Stirling Moss   23.00      2
545   1955       608  Eugenio Castellotti   12.00      3
546   1955       427  Maurice Trintignant   11.33      4
554   1955       642          Nino Farina   10.33      5
...    ...       ...                  ...     ...    ...
3133  2022       855          Guanyu Zhou    6.00     18
3136  2022       848      Alexander Albon    4.00     19
3139  2022       849      Nicholas Latifi    2.00     20
3145  2022       856        Nyck de Vries    2.00     20
3140  2022       807      Nico Hülkenberg    0.00     22

[2652 rows x 5 columns]


In [13]:
#drivers_without_outliers.to_csv("DRIVERS_WITHOUT_OUTLIERS.csv", index=False)

In [5]:
driver_participation = drivers_without_outliers.groupby(['driverId', 'year']).size().reset_index(name='participated')

driver_seasons = driver_participation[driver_participation['participated'] > 0][['driverId', 'year']]

driver_years_list = driver_seasons.groupby('driverId')['year'].apply(list).reset_index(name='seasons_participated')

driver_years_list['number_of_seasons'] = driver_years_list['seasons_participated'].apply(len)

print(driver_years_list)

csv_file_path_reshaped = 'number_of_seasons.csv'
driver_years_list.to_csv(csv_file_path_reshaped, index=False)

     driverId                               seasons_participated  \
0           1  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...   
1           2  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
2           3  [2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...   
3           4  [2001, 2003, 2004, 2005, 2006, 2007, 2008, 200...   
4           5         [2007, 2008, 2009, 2010, 2011, 2012, 2013]   
..        ...                                                ...   
697       852                                       [2021, 2022]   
698       853                                             [2021]   
699       854                                       [2021, 2022]   
700       855                                             [2022]   
701       856                                             [2022]   

     number_of_seasons  
0                   16  
1                   12  
2                   11  
3                   19  
4                    7  
..                 ...  
697     

In [9]:
from drivers_functions import get_drivers
drivers = get_drivers(conn)

     driverId  forename     surname nationality         dob
0           1     Lewis    Hamilton     British  1985-01-07
1           2      Nick    Heidfeld      German  1977-05-10
2           3      Nico     Rosberg      German  1985-06-27
3           4  Fernando      Alonso     Spanish  1981-07-29
4           5    Heikki  Kovalainen     Finnish  1981-10-19
..        ...       ...         ...         ...         ...
852       854      Mick  Schumacher      German  1999-03-22
853       855    Guanyu        Zhou     Chinese  1999-05-30
854       856      Nyck    de Vries       Dutch  1995-02-06
855       857     Oscar     Piastri  Australian  2001-04-06
856       858     Logan    Sargeant    American  2000-12-31

[857 rows x 5 columns]


In [10]:
drivers = pd.merge(drivers, driver_years_list, on='driverId', how='left')

     driverId  forename     surname nationality         dob  \
0           1     Lewis    Hamilton     British  1985-01-07   
1           2      Nick    Heidfeld      German  1977-05-10   
2           3      Nico     Rosberg      German  1985-06-27   
3           4  Fernando      Alonso     Spanish  1981-07-29   
4           5    Heikki  Kovalainen     Finnish  1981-10-19   
..        ...       ...         ...         ...         ...   
852       854      Mick  Schumacher      German  1999-03-22   
853       855    Guanyu        Zhou     Chinese  1999-05-30   
854       856      Nyck    de Vries       Dutch  1995-02-06   
855       857     Oscar     Piastri  Australian  2001-04-06   
856       858     Logan    Sargeant    American  2000-12-31   

                                  seasons_participated  number_of_seasons  
0    [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...               16.0  
1    [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...               12.0  
2    [2006, 200

In [11]:
drivers = drivers.dropna(subset=['seasons_participated', 'number_of_seasons'])

     driverId  forename     surname nationality         dob  \
0           1     Lewis    Hamilton     British  1985-01-07   
1           2      Nick    Heidfeld      German  1977-05-10   
2           3      Nico     Rosberg      German  1985-06-27   
3           4  Fernando      Alonso     Spanish  1981-07-29   
4           5    Heikki  Kovalainen     Finnish  1981-10-19   
..        ...       ...         ...         ...         ...   
850       852      Yuki     Tsunoda    Japanese  2000-05-11   
851       853    Nikita     Mazepin     Russian  1999-03-02   
852       854      Mick  Schumacher      German  1999-03-22   
853       855    Guanyu        Zhou     Chinese  1999-05-30   
854       856      Nyck    de Vries       Dutch  1995-02-06   

                                  seasons_participated  number_of_seasons  
0    [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...               16.0  
1    [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...               12.0  
2    [2006, 200

In [12]:
from lap_times_functions import get_lap_time_statistics
drivers = get_lap_time_statistics(conn, drivers)

     driverId  forename     surname nationality         dob  \
0           1     Lewis    Hamilton     British  1985-01-07   
1           2      Nick    Heidfeld      German  1977-05-10   
2           3      Nico     Rosberg      German  1985-06-27   
3           4  Fernando      Alonso     Spanish  1981-07-29   
4           5    Heikki  Kovalainen     Finnish  1981-10-19   
..        ...       ...         ...         ...         ...   
697       852      Yuki     Tsunoda    Japanese  2000-05-11   
698       853    Nikita     Mazepin     Russian  1999-03-02   
699       854      Mick  Schumacher      German  1999-03-22   
700       855    Guanyu        Zhou     Chinese  1999-05-30   
701       856      Nyck    de Vries       Dutch  1995-02-06   

                                  seasons_participated  number_of_seasons  \
0    [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...               16.0   
1    [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...               12.0   
2    [2006, 

In [16]:
drivers = drivers.dropna()
print(drivers)

     driverId  forename     surname nationality         dob  \
0           1     Lewis    Hamilton     British  1985-01-07   
1           2      Nick    Heidfeld      German  1977-05-10   
2           3      Nico     Rosberg      German  1985-06-27   
3           4  Fernando      Alonso     Spanish  1981-07-29   
4           5    Heikki  Kovalainen     Finnish  1981-10-19   
..        ...       ...         ...         ...         ...   
697       852      Yuki     Tsunoda    Japanese  2000-05-11   
698       853    Nikita     Mazepin     Russian  1999-03-02   
699       854      Mick  Schumacher      German  1999-03-22   
700       855    Guanyu        Zhou     Chinese  1999-05-30   
701       856      Nyck    de Vries       Dutch  1995-02-06   

                                  seasons_participated  number_of_seasons  \
0    [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...               16.0   
1    [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...               12.0   
2    [2006, 

In [18]:
from lap_times_functions import get_lap_positions
drivers = get_lap_positions(conn, drivers)
print(drivers)

     driverId  forename     surname nationality         dob  \
0           1     Lewis    Hamilton     British  1985-01-07   
1           2      Nick    Heidfeld      German  1977-05-10   
2           3      Nico     Rosberg      German  1985-06-27   
3           4  Fernando      Alonso     Spanish  1981-07-29   
4           5    Heikki  Kovalainen     Finnish  1981-10-19   
..        ...       ...         ...         ...         ...   
130       852      Yuki     Tsunoda    Japanese  2000-05-11   
131       853    Nikita     Mazepin     Russian  1999-03-02   
132       854      Mick  Schumacher      German  1999-03-22   
133       855    Guanyu        Zhou     Chinese  1999-05-30   
134       856      Nyck    de Vries       Dutch  1995-02-06   

                                  seasons_participated  number_of_seasons  \
0    [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...               16.0   
1    [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...               12.0   
2    [2006, 

In [19]:
from lap_times_functions import get_total_laps_count
drivers = get_total_laps_count(conn, drivers)
print(drivers)

     driverId  forename     surname nationality         dob  \
0           1     Lewis    Hamilton     British  1985-01-07   
1           2      Nick    Heidfeld      German  1977-05-10   
2           3      Nico     Rosberg      German  1985-06-27   
3           4  Fernando      Alonso     Spanish  1981-07-29   
4           5    Heikki  Kovalainen     Finnish  1981-10-19   
..        ...       ...         ...         ...         ...   
130       852      Yuki     Tsunoda    Japanese  2000-05-11   
131       853    Nikita     Mazepin     Russian  1999-03-02   
132       854      Mick  Schumacher      German  1999-03-22   
133       855    Guanyu        Zhou     Chinese  1999-05-30   
134       856      Nyck    de Vries       Dutch  1995-02-06   

                                  seasons_participated  number_of_seasons  \
0    [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...               16.0   
1    [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...               12.0   
2    [2006, 

In [ ]:
#Question as the pistops were not collected the dataset will be even more reduced to around 70 rows

In [24]:
drivers_w_pistop = drivers.copy()

In [25]:
from pit_stop_functions import get_min_pit_stop_time
drivers_w_pistop = get_min_pit_stop_time(conn, drivers_w_pistop)

In [26]:
from pit_stop_functions import get_avg_pit_time
drivers_w_pistop = get_avg_pit_time(conn, drivers_w_pistop)

In [27]:
from pit_stop_functions import get_max_pit_stop_time
drivers_w_pistop = get_max_pit_stop_time(conn, drivers_w_pistop)

In [28]:
from pit_stop_functions import get_most_common_pitstop_lap
drivers_w_pistop = get_most_common_pitstop_lap(conn, drivers_w_pistop)

In [29]:
from pit_stop_functions import get_avg_pit_stops
drivers_w_pistop = get_avg_pit_stops(conn, drivers_w_pistop)

In [30]:
drivers_w_pistop

,driverId,forename,surname,nationality,dob,seasons_participated,number_of_seasons,max_lap_times,avg_lap_times,min_lap_times,lap_positions_x,lap_positions_y,total_laps,min_pit_stop_time,avg_pit_stop_time,max_pit_stop_time,most_common_pitstop_lap,avg_num_of_pit_stops
0,1,Lewis,Hamilton,British,1985-01-07,"[2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...",16.0,"[{""2007"": 1453884}, {""2008"": 195953}, {""2009"":...","[{""2007"": 90600.86017357763}, {""2008"": 93210.1...","[{""2007"": 72506}, {""2008"": 74159}, {""2009"": 74...","{""2007"": [{""position"": 1, ""position_count"": 32...","{""2007"": [{""position"": 1, ""position_count"": 32...",17683,"{""2011"": 13173, ""2012"": 17598, ""2013"": 17385, ...","{""2011"": 22666.446428571428, ""2012"": 22671.861...","{""2011"": 35688, ""2012"": 31081, ""2013"": 30085, ...","{""2011"": 16, ""2012"": 36, ""2013"": 31, ""2014"": 2...","{""2011"": 2.267857142857143, ""2012"": 1.63888888..."
1,2,Nick,Heidfeld,German,1977-05-10,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",12.0,"[{""2000"": 218816}, {""2001"": 232209}, {""2002"": ...","[{""2000"": 92277.8944}, {""2001"": 85846.86649214...","[{""2000"": 73593}, {""2001"": 71388}, {""2002"": 72...","{""2000"": [{""position"": 17, ""position_count"": 8...","{""2000"": [{""position"": 17, ""position_count"": 8...",9701,"{""2011"": 20796}","{""2011"": 22933.32}","{""2011"": 26281}","{""2011"": 30}","{""2011"": 1.84}"
2,3,Nico,Rosberg,German,1985-06-27,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",11.0,"[{""2006"": 157809}, {""2007"": 191891}, {""2008"": ...","[{""2006"": 89968.74924471299}, {""2007"": 89718.1...","[{""2006"": 74707}, {""2007"": 73159}, {""2008"": 74...","{""2006"": [{""position"": 11, ""position_count"": 1...","{""2006"": [{""position"": 11, ""position_count"": 1...",11161,"{""2011"": 19534, ""2012"": 17894, ""2013"": 17806, ...","{""2011"": 22283.85714285714, ""2012"": 22912.2564...","{""2011"": 30261, ""2012"": 33529, ""2013"": 42432, ...","{""2011"": 12, ""2012"": 20, ""2013"": 10, ""2014"": 3...","{""2011"": 1.9795918367346939, ""2012"": 1.7692307..."
3,4,Fernando,Alonso,Spanish,1981-07-29,"[2001, 2003, 2004, 2005, 2006, 2007, 2008, 200...",19.0,"[{""2001"": 228415}, {""2003"": 178163}, {""2004"": ...","[{""2001"": 92335.86704384725}, {""2003"": 86991.6...","[{""2001"": 74432}, {""2003"": 70526}, {""2004"": 71...","{""2001"": [{""position"": 16, ""position_count"": 1...","{""2001"": [{""position"": 16, ""position_count"": 1...",19155,"{""2011"": 19936, ""2012"": 17570, ""2013"": 18471, ...","{""2011"": 22737.09259259259, ""2012"": 21908.2857...","{""2011"": 31382, ""2012"": 29557, ""2013"": 29800, ...","{""2011"": 16, ""2012"": 15, ""2013"": 30, ""2014"": 1...","{""2011"": 2.0185185185185186, ""2012"": 1.6285714..."
4,5,Heikki,Kovalainen,Finnish,1981-10-19,"[2007, 2008, 2009, 2010, 2011, 2012, 2013]",7.0,"[{""2007"": 1394884}, {""2008"": 185474}, {""2009"":...","[{""2007"": 92458.82991202346}, {""2008"": 93557.7...","[{""2007"": 73998}, {""2008"": 74207}, {""2009"": 74...","{""2007"": [{""position"": 9, ""position_count"": 16...","{""2007"": [{""position"": 9, ""position_count"": 16...",5971,"{""2011"": 20652, ""2012"": 18801, ""2013"": 22646}","{""2011"": 23796.43181818182, ""2012"": 25135.1764...","{""2011"": 44037, ""2012"": 52206, ""2013"": 36333}","{""2011"": 26, ""2012"": 13, ""2013"": 34}","{""2011"": 1.8863636363636365, ""2012"": 2.0, ""201..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,852,Yuki,Tsunoda,Japanese,2000-05-11,"[2021, 2022]",2.0,"[{""2021"": 2240798}, {""2022"": 3148409}]","[{""2021"": 99996.72912801485}, {""2022"": 99573.4...","[{""2021"": 68455}, {""2022"": 70023}]","{""2021"": [{""position"": 14, ""position_count"": 1...","{""2021"": [{""position"": 14, ""position_count"": 1...",2179,"{""2021"": 14943, ""2022"": 18443}","{""2021"": 250616.05, ""2022"": 123340.80434782608}","{""2021"": 2075695, ""2022"": 3011861}","{""2021"": 15, ""2022"": 18}","{""2021"

In [ ]:
csv_file_path_reshaped = '1.csv'
drivers.to_csv(csv_file_path_reshaped, index=False)

In [20]:
from final_scores import calculate_final_constructor_places 
constructors_final_positions = calculate_final_constructor_places(conn)

In [21]:
print(constructors_final_positions)

     year  constructorId constructor_name  points  place
0    2022              9         Red Bull   759.0      1
1    2022              6          Ferrari   554.0      2
2    2022            131         Mercedes   515.0      3
3    2022            214   Alpine F1 Team   173.0      4
4    2022              1          McLaren   159.0      5
..    ...            ...              ...     ...    ...
895  1958            105         Maserati     6.0      5
896  1958             32       Team Lotus     3.0      6
897  1958            125        Connaught     0.0      7
898  1958            127             OSCA     0.0      7
899  1958             95          Porsche     0.0      7

[900 rows x 5 columns]


In [ ]:
drivers_cleaned = drivers.dropna()

In [ ]:
drivers_cleaned

In [ ]:
csv_file_path_reshaped = '2.csv'
drivers_cleaned.to_csv(csv_file_path_reshaped, index=False)